In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

from analystcommunity.write_connection_data_warehouse import runQuery, to_sql

In [4]:
df = pd.read_csv('ALL_SAC_tiers.csv')

In [5]:
# Get the current date in the desired format
current_date = datetime.now().strftime('%Y-%m-%d')

# Add the current date as a new column to the dataframe
df['created_at'] = current_date

In [6]:
df

,city,customer_id,tier,Adjust_CLTV,created_at
0,BOG,-255070,10,18.799587,2023-06-27
1,BOG,-255038,2,1448.358136,2023-06-27
2,BOG,-255037,10,80.757228,2023-06-27
3,BOG,-254992,10,99.245197,2023-06-27
4,BOG,-254982,3,825.766988,2023-06-27
...,...,...,...,...,...
112196,CMX,313257233,10,0.000000,2023-06-27
112197,CMX,313823833,10,15.278731,2023-06-27
112198,CMX,330542475,10,0.000000,2023-06-27
112199,CMX,333970164,10,0.000000,2023-06-27


In [7]:
df2 = df.loc[:,['created_at','city','customer_id','tier']]

In [8]:
df2.columns = ['created_at', 'region_code', 'customer_id', 'cltv']

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112201 entries, 0 to 112200
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   created_at   112201 non-null  object
 1   region_code  112201 non-null  object
 2   customer_id  112201 non-null  int64 
 3   cltv         112201 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 3.4+ MB


In [10]:
def put_products_demand_estimations(CLTV_df: pd.DataFrame, region_code: str, remove_previous_records: bool = False):
    
    if remove_previous_records:
        del_test_sql = f"""

        DELETE FROM
            lnd_ops.customer_cltv
        WHERE
            region_code = '{region_code}'
        """

        runQuery('ops', del_test_sql)
    
    to_sql(vertical='ops', 
            df_params=CLTV_df,
            table='customer_cltv')

In [11]:
for city in ['BOG','BAQ','MDE','CMX','CWB','SPO','BHZ','VCP']:
    put_products_demand_estimations(df2.loc[df2.region_code == city], city, False)